In [1]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install selenium-stealth

# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument("--window-size=1920,1080")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--proxy-server='direct://'")
options.add_argument("--proxy-bypass-list=*")
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument('ignore-certificate-errors')
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.page_load_strategy = 'eager'

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import re
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium_stealth import stealth
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm.auto import tqdm
import numpy as np

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# Buat run di jupyter
ser = Service("C:\chromedriver\chromedriver.exe")
driver = webdriver.Chrome(service=ser, options=options)

stealth(
    driver,
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
    languages = ["en-US", "en"],
    vendor = "Google Inc.",
    platform = "Win32",
    webgl_vendor = "Intel Inc.",
    renderer = "Intel Iris OpenGL Engine",
    fix_hairline = False,
    run_on_insecure_origins = False,
)

In [4]:
#Import Dataset
df_src = pd.read_csv('https://docs.google.com/spreadsheets/d/' +
                   '1Pc5eGQDrs1mMCu55jvSzWcaTZI1or_1INivYJlCSVBk' +
                   '/export?gid=1998583772&format=csv')
df_src.set_index('index', inplace=True)
df_src['url_src'][10]

'http://lpse.its.ac.id/eproc4'

In [5]:
def nilai_evaluasi(index_body, index_j, lst, body, type_head):
    if index_body == index_j:
        if type_head:
            lst.append(body.text)
        else:
            if 'minus' in str(body):
                lst.append(0)
            elif 'check' in str(body):
                lst.append(1)
            elif 'close' in str(body):
                lst.append(-1)
            elif 'img' in str(body):
                lst.append(1)
            else:
                lst.append('')

In [6]:
def get_data_halaman_tender():
    data_tender = BeautifulSoup(driver.page_source, 'lxml')
    # looping to get main data from one url
    for area in tqdm(data_tender.find_all('tr', {'class': ['odd','even']})):
#         detail_paket = area.select_one('p:nth-of-type(2)').text
#         if '2022' in detail_paket or '2021' in detail_paket:
            #Cek duplikasi paket, jika paket sudah ada sebelumnya maka diskip
            kode_paket = area.find('td', class_="sorting_1").get_text()
            detail_paket = area.select_one('p:nth-of-type(2)').text
            tahun = detail_paket.split(' - ')[1]
            if tahun[0] == 'T':
                if kode_paket in kode_paket_list:
                    continue
                else:
                    kode_paket_list.append(kode_paket)
                    get_data_halaman_evaluasi(tahun, kode_paket)
            else:
                raise ValueError('Format Tahun Salah', tahun, kode_paket)
                


def get_data_halaman_evaluasi(tahun, kode_paket):
    add_link = link_lpse.replace("lelang", "")
    link_evaluasi = "{}".format(add_link + "evaluasi/"+ kode_paket +"/hasil")

    driver.get(link_evaluasi)
    time.sleep(1.5)
    data_evaluasi = BeautifulSoup(driver.page_source, 'lxml')
    content_evaluasi = data_evaluasi.find('div', class_="content").text
    if content_evaluasi == '\n':
        pass
    else:
        index_sk = None
        index_sb = None
        index_st = None
        index_k = None
        index_b = None
        index_a = None
        index_t = None
        index_h = None
        index_sh = None
        index_sa = None
        index_pemenang = None
        index_v = None
        index_pk = None
        index_penawaran = None
        index_pt = None
        index_ra = None
        index_hn = None
        index_alasan = None
        head1=[]
        head2=[]
        skor = ['SK','SB','ST','SkorKualifkasi', 'SkorPembuktian','SkorTeknis']

        for tr_head in data_evaluasi.select('thead tr'):
            for th_head in tr_head.select('th')[1:]:
                if th_head.text in skor:
                      head1.append(th_head)
                else:
                      head2.append(th_head)

        for index_th_head, e in enumerate(head1[:]):
            if 'SK' == e.text or 'SkorKualifkasi' == e.text:
                index_sk = index_th_head
            if 'SB' == e.text or 'SkorPembuktian' == e.text:
                index_sb = index_th_head
            if 'ST' == e.text or 'SkorTeknis' == e.text:
                index_st = index_th_head

        for index_td_head, j in enumerate(head2[:]):
            #Cek K B A T H di head halaman
            if 'K' == j.text:
                index_k = index_td_head
                head2.remove(j)
            if 'B' == j.text:
                index_b = index_td_head
                head2.remove(j)
            if 'A' == j.text:
                index_a = index_td_head
                head2.remove(j)
            if 'T' == j.text:
                index_t = index_td_head
                head2.remove(j)
            if 'H' == j.text:
                index_h = index_td_head
                head2.remove(j)
            #Cek SH di head halaman
            if 'SH' == j.text or 'SkorHarga' == j.text:
                index_sh = index_td_head
                head2.remove(j)
            #Cek SA di head halaman
            if 'SA' == j.text or 'SkorAkhir' == j.text:
                index_sa = index_td_head
                head2.remove(j)
            #Cek Pemenang di head halaman
            if 'warning' in str(j):
                index_pemenang = index_td_head
                head2.remove(j)
            if 'V' == j.text:
                index_v = index_td_head
                head2.remove(j)
            if 'PK' == j.text:
                index_pk = index_td_head
                head2.remove(j)
            #Cek Alasan di head halaman
            if 'Alasan' == j.text:
                index_alasan = index_td_head
                head2.remove(j)
            #Cek Harga di head halaman
            if 'Penawaran' == j.text or'<span class="badge badge-success">P</span>' in str(j):
                index_penawaran = index_td_head
                head2.remove(j)
            if 'Terkoreksi' in j.text or '<span class="badge badge-success">PT</span>'  in str(j):
                index_pt = index_td_head
                head2.remove(j)
            if 'Auction' in str(j) or '<span class="badge badge-success">RA</span>'  in str(j):
                index_ra = index_td_head
                head2.remove(j)
            if 'Negosiasi' in str(j) or '<span class="badge badge-success">HN</span>'  in str(j):
                index_hn = index_td_head
                head2.remove(j)

        if len(head2) == 1:
            for tr_body in data_evaluasi.select('tbody tr'):
                #variabel url lpse dan kode paket untuk data evaluasi
                link_lpse_evaluasi_list.append(link_lpse)
                kode_paket_evaluasi_list.append(kode_paket)
                tahun_evaluasi_list.append(tahun)
                
                #variabel npwp
                string_peserta = tr_body.select_one('td:nth-of-type(2)').text
                len_delimiter = string_peserta.count(' - ')
                
                if len_delimiter == 0:
                    npwp = ''
                    nama_peserta = string_peserta.split(' -')[0]
                elif len_delimiter == 1:
                    npwp = string_peserta.split(' - ')[1]
                    nama_peserta = string_peserta.replace(npwp, '').rstrip(' - ')
                elif len_delimiter == 2:
                    npwp = string_peserta.split(' - ')[2]
                    nama_peserta = string_peserta.replace(npwp, '').rstrip(' - ')
                
                replacers ={'o':'0','O':'0'}
                for i, j in replacers.items():
                    npwp = npwp.replace(i, j)
                
                if npwp == '-' or npwp == '':
                    nama_peserta_evaluasi_list.append(nama_peserta)
                    npwp_evaluasi_list.append(npwp)
                elif npwp[0].isdigit():
                    nama_peserta_evaluasi_list.append(nama_peserta)
                    npwp_evaluasi_list.append(npwp)
                else:
                    raise ValueError('Format NPWP salah', npwp, link_evaluasi)

                for index_th_body, th_body in enumerate(tr_body.select('th')):
                    nilai_evaluasi(index_th_body, index_sk, sk_list, th_body, type_head = True)
                    nilai_evaluasi(index_th_body, index_sb, sb_list, th_body, type_head = True)
                    nilai_evaluasi(index_th_body, index_st, st_list, th_body, type_head = True)
                for index_td_body, td_body in enumerate(tr_body.select('td')[1:]):
                    nilai_evaluasi(index_td_body, index_k, k_list, td_body, type_head = False)
                    nilai_evaluasi(index_td_body, index_b, b_list, td_body, type_head = False)
                    nilai_evaluasi(index_td_body, index_a, a_list, td_body, type_head = False)
                    nilai_evaluasi(index_td_body, index_t, t_list, td_body, type_head = False)
                    nilai_evaluasi(index_td_body, index_penawaran, hargapenawaran_list, td_body, type_head = True)
                    nilai_evaluasi(index_td_body, index_pt, hargaterkoreksi_list, td_body, type_head = True)
                    nilai_evaluasi(index_td_body, index_ra, reverseauction_list, td_body, type_head = True)
                    nilai_evaluasi(index_td_body, index_hn, harganegosiasi_list, td_body, type_head = True)
                    nilai_evaluasi(index_td_body, index_h, h_list, td_body, type_head = False)
                    nilai_evaluasi(index_td_body, index_sh, sh_list, td_body, type_head = True)
                    nilai_evaluasi(index_td_body, index_sa, sa_list, td_body, type_head = True)
                    nilai_evaluasi(index_td_body, index_pemenang, list_pemenang, td_body, type_head = False)
                    nilai_evaluasi(index_td_body, index_v, list_pemenang_terverifikasi, td_body, type_head = False)
    
                    if index_td_body == index_pk:
                        if 'img' in str(td_body):
                            list_pemenang_kontrak.append(1)
                            get_data_halaman_pemenang_berkontrak(link_evaluasi, npwp)
                            
                        else:
                            list_pemenang_kontrak.append('')
                            nilai_pdn_list.append('')
                            nilai_umk_list.append('')
                    
                    nilai_evaluasi(index_td_body, index_alasan, alasan_list, td_body, type_head = True)

                if index_sk == None:
                    sk_list.append('')
                if index_sb == None:
                    sb_list.append('')
                if index_st == None:
                    st_list.append('')
                if index_k == None:
                    k_list.append('')
                if index_b == None:
                    b_list.append('')
                if index_a == None:
                    a_list.append('')
                if index_t == None:
                    t_list.append('')
                if index_h == None:
                    h_list.append('')
                if index_sh == None:
                    sh_list.append('')
                if index_sa == None:
                    sa_list.append('')
                if index_pemenang == None:
                    list_pemenang.append('')
                if index_v == None:
                    list_pemenang_terverifikasi.append('')
                    
                if index_pk == None:
                    list_pemenang_kontrak.append('')
                    nilai_pdn_list.append('')
                    nilai_umk_list.append('')
                    
                if index_penawaran == None:
                    hargapenawaran_list.append('')
                if index_pt == None:
                    hargaterkoreksi_list.append('')
                if index_ra == None:
                    reverseauction_list.append('')
                if index_hn == None:
                    harganegosiasi_list.append('')
                if index_alasan == None:
                    alasan_list.append('')
     
        else:
            raise ValueError('Ada head yang tidak terdektesi', link_evaluasi)
        

def get_data_halaman_pemenang_berkontrak(link_evaluasi, npwp):
    link_pemenang_berkontrak = link_evaluasi.replace("hasil", "pemenangberkontrak")
    driver.get(link_pemenang_berkontrak)
#     time.sleep(2)
    data_pemenang_kontrak = BeautifulSoup(driver.page_source, 'lxml')
    content_pk = data_pemenang_kontrak.find('div', class_="content")
    if content_pk == None or content_pk.text == '\n' or content_pk.text == '\n\n\n' :
            nilai_pdn_list.append('')
            nilai_umk_list.append('')
    else:
        index_npwp = None
        index_pdn = None
        index_umk = None
        nilai_pdn = None
        nilai_umk = None
        for index_th, th in enumerate(data_pemenang_kontrak.select('td[colspan="2"] th')):
            if 'NPWP' in th.text:
                index_npwp = index_th
            if 'PDN' in th.text:
                index_pdn = index_th
            if 'UMK' in th.text:
                index_umk = index_th
        body = data_pemenang_kontrak.select('td[colspan="2"] td')
        if body:
            for index_td, td in enumerate(body):
                if index_td == index_npwp:
                    npwp_pk = td.text
                if index_td == index_pdn:
                    nilai_pdn = td.text
                if index_td == index_umk:
                    nilai_umk = td.text

            if index_npwp == None:
                nilai_pdn_list.append('')
                nilai_umk_list.append('')
            elif npwp_pk == npwp:
                if nilai_pdn and index_pdn:
                    nilai_pdn_list.append(nilai_pdn)
                else:
                    nilai_pdn_list.append('')
                if nilai_umk and index_umk:
                    nilai_umk_list.append(nilai_umk)
                else:
                    nilai_umk_list.append('') 
            else:
                nilai_pdn_list.append('')
                nilai_umk_list.append('')
        else:
            nilai_pdn_list.append('')
            nilai_umk_list.append('')
            pass
            
def get_data_filterjumlah(jumlah):
    # Pilih jumlah data per halaman yg ingin ditampilkan
    paginasi = False
    driver.find_element(By.NAME,"tbllelang_length").send_keys(jumlah)
    if jumlah == 'Semua':
        time.sleep(20)
    else:
        paginasi = True
        time.sleep(2)
    # Jumlah paginasi halaman
    page_list = driver.find_elements(By.XPATH, "//ul[@class='pagination']//li")
    last = int(page_list[-3].text)
    for i in range(1,last+1):
        print('looping tender per tahun')
        get_data_halaman_tender()
        if paginasi:
            driver.get(link_lpse)
#             time.sleep(2)
            driver.find_element(By.NAME,"tbllelang_length").send_keys(jumlah)
            time.sleep(2)
            for j in range(i):
                driver.execute_script("arguments[0].click()", driver.find_element(By.XPATH, "//li[@id='tbllelang_next']"))
                time.sleep(2)
        
        
def get_data_filtertahun(tahun, jumlah):
    # get website
    driver.get(link_lpse)
    # waiting to website load succesfully
#     time.sleep(2)
    # Filter tahun sesuai parameter
    Select(driver.find_element(By.XPATH, "//select[@name='tahun']")).select_by_visible_text(tahun)
    time.sleep(2)
    get_data_filterjumlah(jumlah)

In [7]:
err_link_peserta = []
err_link_lpse = []

In [45]:
#variabel data tender
kode_paket_list = []

#variabel data evaluasi
link_lpse_evaluasi_list = []
kode_paket_evaluasi_list = []
tahun_evaluasi_list = []
nama_peserta_evaluasi_list = []
npwp_evaluasi_list=[]
sk_list =[]
sb_list =[]
st_list =[]
k_list = []
b_list =[]
a_list =[]
t_list = []
h_list =[]
hargapenawaran_list =[]
hargaterkoreksi_list =[]
harganegosiasi_list=[]
reverseauction_list =[]
list_pemenang = []
list_pemenang_terverifikasi =[]
list_pemenang_kontrak = []
sh_list = []
sa_list = []
alasan_list=[]

#Variabel data pemenang berkontrak
nilai_pdn_list = []
nilai_umk_list = []

In [46]:
# set Url based on index in dataset
i=46
last=47

# Looping to put data in variables
for i in tqdm(range(i, last)):
    try:
        # url from dataset add /lelang
        link_lpse = "{}".format(df_src['url_src'][i]+"/lelang")

        driver.set_window_size(1920,1080)

        # get website
        driver.get(link_lpse)
        # waiting to website load succesfully
#         time.sleep(2)
        tampilan = driver.find_elements(By.XPATH, "//select[@name='tbllelang_length']//option")
        data_home_default = BeautifulSoup(driver.page_source, 'lxml')
        years = [e.text for e in data_home_default.select('select[name="tahun"] option')[1:-1]]
        years.sort(reverse=True)
        years = years[2:4]
        if years == [] or len(tampilan) < 4 :
            get_data_filterjumlah('100')
        else:
            print('looping per tahun')
            for year in tqdm(years):
                get_data_filtertahun(year, 'Semua')


    except:
        # next index url
        err_link_lpse.append(link_lpse)
        pass

  0%|          | 0/1 [00:00<?, ?it/s]

looping per tahun


  0%|          | 0/2 [00:00<?, ?it/s]

looping tender per tahun


  0%|          | 0/96 [00:00<?, ?it/s]

looping tender per tahun


  0%|          | 0/296 [00:00<?, ?it/s]

In [47]:
err_link_lpse

['http://lpse.sangihekab.go.id/eproc4/lelang',
 'http://lpse.gowakab.go.id/eproc4/lelang',
 'http://lpse.gowakab.go.id/eproc4/lelang',
 'http://lpse.gowakab.go.id/eproc4/lelang',
 'http://lpse.gowakab.go.id/eproc4/lelang']

In [48]:
data_evaluasi_dict = {'url_lpse':link_lpse_evaluasi_list, 'kode_paket':kode_paket_evaluasi_list,
    'tahun':tahun_evaluasi_list,
    'nama_peserta':nama_peserta_evaluasi_list, 'npwp':npwp_evaluasi_list,
    'kualifikasi':k_list, 'skor_kualifikasi':sk_list,
    'pembuktian_kualifikasi':b_list, 'skor_pembuktian':sb_list,
    'administrasi':a_list,
    'teknis':t_list, 'skor_teknis':st_list,
    'penawaran':hargapenawaran_list, 'penawaran_terkoreksi':hargaterkoreksi_list, 'reverse_auction':reverseauction_list, 'hasil_negosiasi':harganegosiasi_list,
    'evaluasi_harga':h_list,'skor_harga':sh_list,
    'skor_akhir':sa_list,
    'pemenang':list_pemenang, 'pemenang_terverifikasi':list_pemenang_terverifikasi, 'pemenang_berkontrak':list_pemenang_kontrak, 
    'nilai_pdn':nilai_pdn_list, 'nilai_umk':nilai_umk_list,
    'alasan':alasan_list}

In [49]:
import pandas as pd
df_evaluasi = pd.DataFrame.from_dict(data_evaluasi_dict, orient = 'index').T
pd.options.display.max_columns = 50
# pd.set_option('display.max_rows', None)
df_evaluasi

,url_lpse,kode_paket,tahun,nama_peserta,npwp,kualifikasi,skor_kualifikasi,pembuktian_kualifikasi,skor_pembuktian,administrasi,teknis,skor_teknis,penawaran,penawaran_terkoreksi,reverse_auction,hasil_negosiasi,evaluasi_harga,skor_harga,skor_akhir,pemenang,pemenang_terverifikasi,pemenang_berkontrak,nilai_pdn,nilai_umk,alasan
0,https://lpse.banyuwangikab.go.id/eproc4/lelang,13594072,TA 2020,CV. Harrisma Computer,02.961.108.4-541.000,,,,,,,,"Rp. 795.999.930,00","Rp. 795.999.930,00",,,1,,,,,,,,
1,https://lpse.banyuwangikab.go.id/eproc4/lelang,13594072,TA 2020,CV. BOLU AGUNG,01.203.346.0-651.000,,,,,,,,,,,,,,,,,,,,
2,https://lpse.banyuwangikab.go.id/eproc4/lelang,13594072,TA 2020,CV. HEBRING INTERTEK,85.633.328.1-005.000,,,,,,,,,,,,,,,,,,,,
3,https://lpse.banyuwangikab.go.id/eproc4/lelang,13575072,TA 2020,CV. Prima Perkasa Engineering,02.909.652.6-606.000,,,,,,,,"Rp. 196.638.750,00 di bawah 80% Nilai HPS","Rp. 196.638.750,00 di bawah 80% Nilai HPS",,,1,,,1,1,,,,
4,https://lpse.banyuwangikab.go.id/eproc4/lelang,13575072,TA 2020,CV. KARTIKA SARI,01.497.686.4-615.000,,,,,,,,"Rp. 205.065.000,00 di bawah 80% Nilai HPS","Rp. 205.065.000,00 di bawah 80% Nilai HPS",,,1,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15334,https://lpse.banyuwangikab.go.id/eproc4/lelang,3198072,TA 2019,CV. KARSA GEMILANG,02.267.742.1-627.000,0,,0,,0,0,,,,,,0,,,,,,,,
15335,https://lpse.banyuwangikab.go.id/eproc4/lelang,3197072,TA 2019,KOPERASI PENGEMUDI KARYA NYATA,67.974.176.9-627.000,1,,1,,1,1,,"Rp. 299.996.400,00","Rp. 299.996.400,00",,"Rp. 299.640.000,00",1,,,1,,,,,
15336,https://lpse.banyuwangikab.go.id/eproc4/lelang,3197072,TA 2019,CV WARU MAS,01.232.730.0-609.000,0,,0,,0,0,,,,,,0,,,,,,,,
15337,https://lpse.banyuwangikab.go.id/eproc4/lelang,3197072,TA 2019,CV. KARSA GEMILANG,02.267.742.1-627.000,0,,0,,0,0,,,,,,0,,,,,,,,


In [50]:
 with pd.ExcelWriter(f"D:/Data Engineer - Beecons/Document/hasil evaluasi/Evaluasi_{i}_{years}.xlsx", options={'strings_to_urls': False}) as writer:
        df_evaluasi.to_excel(writer, index=False)

C:\Users\Asus\AppData\Local\Temp/ipykernel_8064/3649372524.py:1: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  with pd.ExcelWriter(f"D:/Data Engineer - Beecons/Document/hasil evaluasi/Evaluasi_{i}_{years}.xlsx", options={'strings_to_urls': False}) as writer:


In [21]:
len(kode_paket_list)

1668

In [46]:
df_evaluasi['kode_paket'].nunique()

1354